<a href="https://colab.research.google.com/github/Roopesh-Bharatwaj-K-R/Computer-Vision_Robotic-Arm/blob/Master/On_device_decoding_Ball__Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This File tells how to convert the trained Yolov5 Model to Deploymnet of Model in OAK Camera.

Mount Drive 

In [ ]:
# save file
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
path = "/content/gdrive/MyDrive/yolov5" # /content is pretty much the root. you can choose other path in your colab workspace
os.chdir(path)

# OpenVino :

In [ ]:
%%time
%%capture
## install tools. Open Vino takes some time to download: 10-15 min sometimes.
!sudo apt-get install -y pciutils cpio
!sudo apt autoremove
## downnload installation files
#! wget http://registrationcenter-download.intel.com/akdlm/irc_nas/16612/l_openvino_toolkit_p_2020.2.120.tgz
! wget https://github.com/PINTO0309/tflite2tensorflow/releases/download/v1.10.4/l_openvino_toolkit_p_2021.4.582.tgz
path = "l_openvino_toolkit_p_2021.4.582.tgz"

## install openvino
!tar xf "{path}"
%cd l_openvino_toolkit_p_2021.4.582/

!./install_openvino_dependencies.sh && \
    sed -i 's/decline/accept/g' silent.cfg && \
    ./install.sh --silent silent.cfg

CPU times: user 1.58 s, sys: 320 ms, total: 1.9 s
Wall time: 3min 12s


In [ ]:
!bash /opt/intel/openvino_2021/deployment_tools/model_optimizer/install_prerequisites/install_prerequisites.sh

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

# Import the Yolov5 Model Weight

In [ ]:
weights_path = "/content/gdrive/MyDrive/yolov5/runs/train/exp/weights/firstbest.pt"

# Convert Yolov5 weights.pt to Onnx 

This code saves the Model as '.onnx' 

In [ ]:
! python export.py --weights $weights_path --img 416 --batch 1 --device cpu --include "onnx" --simplify 

export: data=data/coco128.yaml, weights=['/content/gdrive/MyDrive/yolov5/runs/train/exp/weights/firstbest.pt'], imgsz=[416], batch_size=1, device=cpu, half=False, inplace=False, train=False, optimize=False, int8=False, dynamic=False, simplify=True, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v6.1-69-g7830e91 torch 1.10.0+cu111 CPU

Fusing layers... 
Model summary: 213 layers, 7023610 parameters, 0 gradients

PyTorch: starting from /content/gdrive/MyDrive/yolov5/runs/train/exp/weights/firstbest.pt with output shape (1, 10647, 10) (13.7 MB)

ONNX: starting export with onnx 1.11.0...
ONNX: simplifying with onnx-simplifier 0.3.7...
ONNX: export success, saved as /content/gdrive/MyDrive/yolov5/runs/train/exp/weights/firstbest.onnx (27.0 MB)

Export complete (4.84s)
Results saved to /content/gdrive/MyDrive/yolov5/runs/train/exp/weights
Detect:          python detect.py --weig

In [ ]:
!pip install onnx

Import Onnx path 

In [ ]:
onnx_path = '/content/gdrive/MyDrive/yolov5/runs/train/exp/weights/firstbest.onnx'

# Pruning the Last 3 layers of Onnx Model (Conv layer  to Sigmoid Layer)

Use Netron App for the Viewing the LAYERS and Pruning.

In [ ]:
import onnx

onnx_model = onnx.load(onnx_path)

conv_indices = []
for i, n in enumerate(onnx_model.graph.node):
  if "Conv" in n.name:
    conv_indices.append(i)

In [ ]:
input1, input2, input3 = conv_indices[-3:]

In [ ]:
sigmoid1 = onnx.helper.make_node(
    'Sigmoid',
    inputs=[onnx_model.graph.node[input1].output[0]],
    outputs=['output1_yolov5'],
)

sigmoid2 = onnx.helper.make_node(
    'Sigmoid',
    inputs=[onnx_model.graph.node[input2].output[0]],
    outputs=['output2_yolov5'],
)

sigmoid3 = onnx.helper.make_node(
    'Sigmoid',
    inputs=[onnx_model.graph.node[input3].output[0]],
    outputs=['output3_yolov5'],
)

onnx_model.graph.node.append(sigmoid1)
onnx_model.graph.node.append(sigmoid2)
onnx_model.graph.node.append(sigmoid3)

In [ ]:
! mkdir  "/content/gdrive/MyDrive/yolov5/pruned_onnx/"
#onnx.save(onnx_model, "/content/gdrive/MyDrive/yolov5/pruned_onnx/pruned.onnx")

mkdir: cannot create directory ‘/content/gdrive/MyDrive/yolov5/pruned_onnx/’: File exists


# Save Model

In [ ]:
onnx.save(onnx_model, "/content/gdrive/MyDrive/yolov5/pruned_onnx/pruned.onnx")

All that is left to do, is to export the model to OpenVINO IR and generate the blob that we can use together with the DepthAI API! 💻

If you face some erros regarding the input shape, try adding the flag --input_shape [1,3,416,416] with correct shape.

Note that we name the output layers as we named them when editing the ONNX. As this are our output layers, this will also remove all later layers and prune our model.

# OpenVINO  Model Optimizer 

The code optimizes as OpenVINO IR.

yolov5.xml and yolov5.bin saved in the output directory. 

We can use them now to generate the .blob that can run on OAK.

In [ ]:
! source /opt/intel/openvino_2021/bin/setupvars.sh
! python /opt/intel/openvino_2021/deployment_tools/model_optimizer/mo.py \
--input_model  /content/gdrive/MyDrive/yolov5/pruned_onnx/pruned.onnx \
--model_name ball_detector_yolov5 \
--data_type FP16 \
--output_dir /content/gdrive/MyDrive/yolov5/output \ # Output Directory
--reverse_input_channel \
--scale 255 \
--output "output1_yolov5,output2_yolov5,output3_yolov5"

[setupvars.sh] OpenVINO environment initialized
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/content/gdrive/MyDrive/yolov5/pruned_onnx/pruned.onnx
	- Path for generated IR: 	/content/gdrive/MyDrive/yolov5/output
	- IR output name: 	ball_detector_yolov5
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	output1_yolov5,output2_yolov5,output3_yolov5
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	255.0
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	True
ONNX specific parameters:
	- Inference Engine found in: 	/opt/intel/openvino_2021.4.582/python/python3.7/openvino
Inference Engine version: 	2021.4.0-3839-cd81789d294-releases/2021/4
Model Optimizer v

# OpenVino to Blob Converter

This Blob File can be used directly to the OAK-D-Litte (Intel VPU ) camera for deployment

In [ ]:
! mkdir "/content/gdrive/MyDrive/yolov5/blob_file/" # create a directory 

Import the Yolov5.Bin and Yolov5.xml directory

In [ ]:

blob_dir = "/content/gdrive/MyDrive/yolov5/blob_file/"  # Choose the directory where you would like to save the blob.

binfile = "/content/gdrive/MyDrive/yolov5/output/ball_detector_yolov5.bin"
xmlfile = "/content/gdrive/MyDrive/yolov5/output/ball_detector_yolov5.xml"

!python -m pip install blobconverter

import blobconverter
blob_path = blobconverter.from_openvino(
    xml=xmlfile,
    bin=binfile,
    data_type="FP16",
    shaves=6,
    version="2021.4",
    use_cache=False)
from google.colab import files
files.download(blob_path)

# Saved Blob File



# Next Step- > Depthai Pipeline and Deploymemt in OAK Camera